In [38]:
import pickle
import numpy as np
import pandas as pd
import mediapipe as mp # Import mediapipe
import cv2 # Import opencv
from sklearn.metrics import accuracy_score

from collections import deque

In [39]:
mp_drawing = mp.solutions.drawing_utils # Drawing helpers
mp_holistic = mp.solutions.holistic # Mediapipe Solutions

In [40]:
with open('four_poses.pkl', 'rb') as f:
    model = pickle.load(f)

In [41]:
model

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('randomforestclassifier', RandomForestClassifier())])

In [42]:
def calculate_angle(a,b,c):
    a = np.array(a) # First
    b = np.array(b) # Mid
    c = np.array(c) # End
    
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    
    if angle >180.0:
        angle = 360-angle
        
    return angle 

In [48]:
cap = cv2.VideoCapture(0)
pose_container = deque(maxlen=2)
stage1 = [bend_foward, stand]
stage2 = [crouch, bend_foward]
stage3 = [stand, crouch]

stages = [False, False, False]

# Initiate holistic model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor Feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False        
        
        # Make Detections
        results = holistic.process(image)

        
        # Recolor image back to BGR for rendering
        image.flags.writeable = True   
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        
        # Pose Detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )
        
        # Displaying the output
        
        try:
            # Extract Pose landmarks
            pose = results.pose_landmarks.landmark
            pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose]).flatten())
            
            row = pose_row
                            
            # Make Detections
            X = pd.DataFrame([row])
            body_language_class = model.predict(X)[0]
            body_language_prob = model.predict_proba(X)[0]
            
            #print(body_language_class, body_language_prob)
            
            
            
            
            #test code for pose detection
            
            pose_container.append(body_language_class)
            '''
            if pose_container == stage1:
                stages[0] = True
            
            '''
            
            
            
            
            #Calculate and Display angle
            
            # Get right side coordinates
            right_shoulder = [pose[mp_holistic.PoseLandmark.RIGHT_SHOULDER.value].x,pose[mp_holistic.PoseLandmark.RIGHT_SHOULDER.value].y]
            right_hip = [pose[mp_holistic.PoseLandmark.RIGHT_HIP.value].x,pose[mp_holistic.PoseLandmark.RIGHT_HIP.value].y]
            right_knee = [pose[mp_holistic.PoseLandmark.RIGHT_KNEE.value].x,pose[mp_holistic.PoseLandmark.RIGHT_WRIST.value].y]
            
            # Calculate angle
            right_angle = calculate_angle(right_shoulder, right_hip, right_knee)
            
            # Visualize angle
            cv2.putText(image, str(round(right_angle, 2)), 
                           tuple(np.multiply(right_hip, [640, 480]).astype(int)), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA           
                       )
                        
                        
                        
            # Get left side coordinates
            left_shoulder = [pose[mp_holistic.PoseLandmark.LEFT_SHOULDER.value].x,pose[mp_holistic.PoseLandmark.LEFT_SHOULDER.value].y]
            left_hip = [pose[mp_holistic.PoseLandmark.LEFT_HIP.value].x,pose[mp_holistic.PoseLandmark.LEFT_HIP.value].y]
            left_knee = [pose[mp_holistic.PoseLandmark.LEFT_KNEE.value].x,pose[mp_holistic.PoseLandmark.LEFT_WRIST.value].y]
            
            # Calculate angle
            left_angle = calculate_angle(left_shoulder, left_hip, left_knee)
            
            # Visualize angle
            cv2.putText(image, str(round(left_angle, 2)), 
                           tuple(np.multiply(left_hip, [640, 480]).astype(int)), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
                                )
            
            
            
            
            # Get status box
            cv2.rectangle(image, (0,0), (300, 50), (245, 117, 16), -1)
            cv2.rectangle(image, (640,0), (540, 50), (245, 117, 16), -1)
            
            # Display Pose
            cv2.putText(image, 'CLASS'
                        , (95,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            cv2.putText(image, body_language_class.split(' ')[0]
                        , (90,40), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2, cv2.LINE_AA)
            
            # Display Probability
            cv2.putText(image, 'PROB'
                        , (15,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            cv2.putText(image, str(round(body_language_prob[np.argmax(body_language_prob)],2))
                        , (10,40), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2, cv2.LINE_AA)

        except:
            pass

                        
        cv2.imshow('Raw Webcam Feed', cv2.flip(image, 1))
        cv2.imshow('Raw Webcam Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

In [49]:
pose_container

deque(['stand', 'stand', 'stand'])